In [1]:
from pathlib import Path
import numpy as np
import nibabel as nib
import torch
import pandas as pd
import re 

In [20]:
def get_params(params_dir, view):
    df = None
    params_files = list(params_dir.glob('{view}_params*.pt'))
    for fl in hla_params_files:
        
        mt = re.match(r".*(train|test).*([0-9]{1,3})\.pt", str(fl))
        phase, epoch = mt[1], int(mt[2])
        param_dict = torch.load(fl)
        epx_theta_aps = param_dict[f'epx_{view}_theta_aps']
        epx_theta_t_offsets = param_dict[f'epx_{view}_theta_t_offsets']
        epx_theta_zps = param_dict[f'epx_{view}_theta_zps']

        ids = list(zip(*sorted(epx_theta_aps.items())))[0]
        theta_ap = list(zip(*sorted(epx_theta_aps.items())))[1]
        theta_t_offsets = list(zip(*sorted(epx_theta_t_offsets.items())))[1]
        theta_zp = list(zip(*sorted(epx_theta_zps.items())))[1]

        data = dict(
            view=view, 
            sample=ids,
            epoch=epoch,
            theta_ap=[p.detach() for p in theta_ap],
            theta_t_offsets=[p.detach() for p in theta_t_offsets],
            theta_zp=[p.detach() for p in theta_zp],
            phase=phase,
        )

        if df is None:
            df = pd.DataFrame(data)
        else:
            df = pd.concat([df, pd.DataFrame(data)])

    return df

In [21]:
params_dir = Path("/shared/slice_inflate/data/output/20230317__13_53_36_fallen-leaf-903-stage-1")
df = get_params(params_dir, 'sa')
df

,view,sample,epoch,theta_ap,theta_t_offsets,theta_zp,phase
0,sa,1004-mr,0,"[tensor(-1.4763), tensor(-1.3135), tensor(-1.1...","[tensor(0.0032), tensor(0.0319), tensor(-0.0339)]",[tensor(1.0938)],train
1,sa,1007-mr,0,"[tensor(-0.7566), tensor(-0.6392), tensor(-0.4...","[tensor(0.0223), tensor(-0.0051), tensor(-0.00...",[tensor(1.1411)],train
2,sa,1008-mr,0,"[tensor(-1.6121), tensor(-1.3291), tensor(-1.0...","[tensor(0.0058), tensor(0.0326), tensor(-0.0367)]",[tensor(1.1055)],train
3,sa,1009-mr,0,"[tensor(-0.7468), tensor(-1.1797), tensor(0.50...","[tensor(0.0261), tensor(0.0044), tensor(-0.0092)]",[tensor(0.9847)],train
4,sa,1010-mr,0,"[tensor(-1.6033), tensor(-1.3564), tensor(-1.0...","[tensor(0.0080), tensor(0.0335), tensor(-0.0348)]",[tensor(1.2721)],train
...,...,...,...,...,...,...,...
0,sa,1001-mr,9,"[tensor(9.0178), tensor(1.0664), tensor(-1.163...","[tensor(-0.1152), tensor(0.1797), tensor(-0.05...",[tensor(0.7781)],test
1,sa,1002-mr,9,"[tensor(8.9006), tensor(1.0820), tensor(-1.073...","[tensor(-0.1145), tensor(0.1797), tensor(-0.05...",[tensor(1.1013)],test
2,sa,1003-mr,9,"[tensor(8.9241), tensor(0.9282), tensor(-1.134...","[tensor(-0.1145), tensor(0.1797), tensor(-0.05...",[tensor(1.1993)],test
3,sa,1005-mr,9,"[tensor(9.0881), tensor(1.0723), tensor(-1.377...","[tensor(-0.1152), tensor(0.1797), tensor(-0.05...",[tensor(0.7891)],test


In [23]:
newdf = df.query('sample == "1004-mr" & view == "sa"')
newdf

,view,sample,epoch,theta_ap,theta_t_offsets,theta_zp,phase
0,sa,1004-mr,0,"[tensor(-1.4763), tensor(-1.3135), tensor(-1.1...","[tensor(0.0032), tensor(0.0319), tensor(-0.0339)]",[tensor(1.0938)],train
0,sa,1004-mr,1,"[tensor(-0.6975), tensor(0.5059), tensor(1.745...","[tensor(0.0424), tensor(-0.0070), tensor(-0.00...",[tensor(0.7143)],train
0,sa,1004-mr,2,"[tensor(-1.8963), tensor(0.3025), tensor(0.598...","[tensor(0.0897), tensor(0.0191), tensor(0.0199)]",[tensor(0.7163)],train
0,sa,1004-mr,3,"[tensor(-1.8884), tensor(0.1042), tensor(0.221...","[tensor(0.0782), tensor(-0.0082), tensor(0.0737)]",[tensor(0.7765)],train
0,sa,1004-mr,4,"[tensor(-2.1326), tensor(0.4729), tensor(-0.40...","[tensor(0.0429), tensor(-0.0406), tensor(0.1064)]",[tensor(1.2542)],train
...,...,...,...,...,...,...,...
0,sa,1004-mr,5,"[tensor(8.8850), tensor(1.1465), tensor(-1.382...","[tensor(-0.1156), tensor(0.1797), tensor(-0.05...",[tensor(1.0026)],train
0,sa,1004-mr,6,"[tensor(8.9475), tensor(1.2031), tensor(-1.531...","[tensor(-0.1156), tensor(0.1797), tensor(-0.05...",[tensor(0.8267)],train
0,sa,1004-mr,7,"[tensor(8.9944), tensor(1.1758), tensor(-1.570...","[tensor(-0.1156), tensor(0.1797), tensor(-0.05...",[tensor(0.8148)],train
0,sa,1004-mr,8,"[tensor(9.1038), tensor(1.0723), tensor(-1.567...","[tensor(-0.1156), tensor(0.1797), tensor(-0.05...",[tensor(0.8423)],train


In [2]:
import pymeshfix as mf
import pyvista as pv

assert False, "Caution: pyvista plots require a machine with attached physical display or a display emulation (Xvfb package) - otherwise the kernel may die when generating renderings."

ModuleNotFoundError: No module named 'pyvista'

# Create mesh from shape voxels

In [ ]:
import numpy as np
import torch
import h5py
import pymeshfix as mf
import nibabel as nib
import pyvista as pv
from pathlib import Path
from skimage import measure

In [ ]:
def replace_label_values(label):
    # Replace label numbers with MMWHS equivalent
    # STRUCTURE           MMWHS   ACDC    NNUNET
    # background          0       0       0
    # left_myocardium     205     2       1
    # left_atrium         420     N/A     2
    # ?                   421     N/A     N/A
    # left_ventricle      500     3       3
    # right_atrium        550     N/A     4
    # right_ventricle     600     1       5
    # ascending_aorta     820     N/A     6
    # pulmonary_artery    850     N/A     7
    orig_values = [0,  205, 420, 421, 500, 550, 600, 820, 850]
    new_values = [0,  1,   2,   0,   3,   4,   5,   0,   0]

    modified_label = label.clone()
    for orig, new in zip(orig_values, new_values):
        modified_label[modified_label == orig] = new
    return modified_label

In [ ]:
CASE = "1004"

In [ ]:
nii_shape = nib.load(f"mr_train_{CASE}_label_registered.nii.gz")
shape_data = replace_label_values(torch.as_tensor(nii_shape.get_fdata())).long()
shape_affine = torch.as_tensor(nii_shape.affine)
image_sample = nib.load(f"mr_train_{CASE}_image_registered.nii.gz").get_fdata()
SPACING = (1.,1.,1.)
STEP_SIZE = 2
CLASSES = ['background', 'MYO', 'LA', 'LV', 'RA', 'RV']

heart_data = {}
for class_idx, tag in enumerate(CLASSES):
    if class_idx == 0: continue

    sub_label = torch.nn.functional.one_hot(shape_data.long(), len(CLASSES))[:,:,:, class_idx]
    verts, faces, normals, values = measure.marching_cubes(sub_label.cpu().numpy(), spacing=SPACING, step_size=STEP_SIZE)
    mm_verts = torch.cat([torch.tensor(verts.copy()), torch.ones(len(verts),1)], dim=1)
    mm_verts = (shape_affine @ mm_verts.T.double()).T[:,:3]

    data = dict(
        verts=torch.as_tensor(mm_verts.numpy().copy()),
        faces=torch.as_tensor(faces.copy()),
        normals=torch.as_tensor(normals.copy()), 
        values=torch.as_tensor(values.copy())
    )
    heart_data[tag] = data

In [55]:
import re
optimized_sa_affines = []

run_data_path = Path("/shared/slice_inflate/data/output/20230310__01_40_03_devout-brook-811-stage-1")

file_paths = list(run_data_path.glob('*.pt'))

key = lambda _path: int(re.match(r".*?([0-9]+)\.pt", str(_path))[1])
file_paths.sort(key=key)

for fl in file_paths:
    content = torch.load(fl)
    content.keys()
    break

In [50]:
mt = int(re.match(r".*?([0-9]+)\.pt", str(file_paths))[1])
print(mt)

0


In [ ]:
hla_mat = torch.from_numpy(np.loadtxt("mmwhs_1002_4CH.mat"))
sa_mat = torch.from_numpy(np.loadtxt("mmwhs_1002_SA.mat"))

In [ ]:
optimized_sa_mat_1004 = torch.tensor(
    [[[ 0.3762,  0.0235, -0.9262,  0.0000],
      [ 0.6500,  0.7056,  0.2820,  0.0000],
      [ 0.6602, -0.7082,  0.2502,  0.0000],
      [ 0.0000,  0.0000,  0.0000,  1.0000]]]
)

optimized_hla_mat_1004 = torch.tensor(
    [[[ 0.8096, -0.5865,  0.0236,  0.0000],
      [ 0.5751,  0.7844, -0.2325,  0.0000],
      [ 0.1178,  0.2018,  0.9723,  0.0000],
      [ 0.0000,  0.0000,  0.0000,  1.0000]]]
)

In [ ]:
from align_mmwhs import nifti_transform
FOV_MM = torch.tensor([224,224,224])
FOV_VOX = torch.tensor([160,160,160])

with torch.no_grad():
    sa_label, sa_affine, sa_grid_affine = nifti_transform(shape_data.unsqueeze(0).unsqueeze(0), shape_affine.unsqueeze(0), sa_mat.unsqueeze(0), 
        fov_mm=FOV_MM, fov_vox=FOV_VOX, is_label=True, 
        # pre_grid_sample_affine=optimized_sa_mat_1004,
        pre_grid_sample_affine=None
    )
    
    hla_label, hla_affine, hla_grid_affine = nifti_transform(shape_data.unsqueeze(0).unsqueeze(0), shape_affine.unsqueeze(0), hla_mat.unsqueeze(0), 
        fov_mm=FOV_MM, fov_vox=FOV_VOX, is_label=True, 
        # pre_grid_sample_affine=optimized_hla_mat_1004,
        pre_grid_sample_affine=None
    )


In [ ]:
from matplotlib import pyplot as plt
plt.imshow(sa_label.squeeze()[:,:,80].T)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(hla_label.squeeze()[:,:,80].T)

In [ ]:
# Nifti original plane slicing.
# sa_normal = torch.tensor([1.,0.,0.])  # working! for D slicing
# sa_normal = torch.tensor([0.,1.,0.])  # working! for H slicing

sa_normal = torch.tensor([0.,0.,1.])  # working! for W slicing
sa_support = (shape_affine @ torch.tensor([64.,64.,64.,1.]).double())[:3]
print("Non transformed", sa_normal, sa_support)

# SA slicing
sa_normal = (sa_affine @ torch.tensor([0.,0.,1.,0.]).double())[0,:3]
sa_support = (sa_affine @ torch.tensor([80.,80.,80.,1.]).double())[0,:3]

print("Transformed SA", sa_normal, sa_support)

hla_normal = (hla_affine @ torch.tensor([0.,0.,1.,0.]).double())[0,:3]
hla_support =(hla_affine @ torch.tensor([80.,80.,80.,1.]).double())[0,:3]
print("Transformed HLA", hla_normal, hla_support)

In [ ]:
h_vertices = heart_data['MYO']['verts']
h_faces = heart_data['MYO']['faces']

for tag, data in heart_data.items():
    faces = data['faces']
    num_faces = faces.shape[0]
    num_points = torch.tensor([3]*num_faces).view(num_faces,1)
    data['pyvista_faces'] = torch.cat([num_points, faces], dim=1)

In [ ]:
# https://coolors.co/b8336a-726da8-7d8cc4-a0d2db-c490d1

palette = [
    '#B8336A',
    '#726DA8',
    '#7D8CC4',
    '#A0D2DB',
    '#C490D1',
]
dark_palette = [
    '#4F172E', 
    '#424064',
    '#485070',
    '#547378',
    '#73507C',
]

In [ ]:
SHOW_FULL_MESH = False

SHOW_SA_PLANE = True
SHOW_SA_SLICE = True

SHOW_HLA_PLANE = True
SHOW_HLA_SLICE = True

CPOS = [
    (456.42537425592207, -476.3133591268548, 11.507223342275584),
    (-49.71246948463747, 0.5437890350509349, -5.752938139380589),
    (0.6710467014241088, 0.7038135452178427, -0.23311546082513432)
]

plotter = pv.Plotter(
    lighting='three lights'
)
plotter.background_color = "white"

cpos = 'iso'
sa_plane = pv.Plane(center=sa_support.tolist(), direction=sa_normal.tolist(), i_size=200, j_size=200, i_resolution=1, j_resolution=1)
sa_plane.point_data.clear()
sa_edges = sa_plane.extract_feature_edges(boundary_edges=True, feature_edges=False, manifold_edges=False)

hla_plane = pv.Plane(center=hla_support.tolist(), direction=hla_normal.tolist(), i_size=200, j_size=200, i_resolution=1, j_resolution=1)
hla_plane.point_data.clear()
hla_edges = hla_plane.extract_feature_edges(boundary_edges=True, feature_edges=False, manifold_edges=False)

full_mesh = []

# Prepare meshes
for idx, (tag, data) in enumerate(heart_data.items()):
    surf = pv.PolyData(data['verts'].numpy(), data['pyvista_faces'].view(-1).numpy())
    scalars=np.array([idx]*data['verts'].shape[0])
    surf.point_data.set_scalars(scalars, 'scalars')
    # full_mesh.append(surf)
    smooth = surf.smooth_taubin(n_iter=100, pass_band=0.3)
    full_mesh.append(smooth)

block = pv.MultiBlock(full_mesh)
full_mesh = block.combine(merge_points=False)

sa_slice = full_mesh.slice(normal=sa_normal.tolist(), origin=sa_support.tolist())
hla_slice = full_mesh.slice(normal=hla_normal.tolist(), origin=hla_support.tolist())

if SHOW_FULL_MESH:
    plotter.add_mesh(full_mesh, name='all', cmap=palette, line_width=2, show_scalar_bar=False, smooth_shading=True)   

if SHOW_SA_SLICE:
    plotter.add_mesh(sa_slice, name=tag +'_sa_slice', cmap=palette, line_width=2, show_scalar_bar=False) 

if SHOW_SA_PLANE:   
    plotter.add_mesh(sa_plane, color=palette[idx],  opacity=0.3, show_edges=False, line_width=2)
    plotter.add_mesh(sa_edges, color=dark_palette[idx], line_width=1)

if SHOW_HLA_SLICE:
    plotter.add_mesh(hla_slice, name=tag +'_hla_slice', cmap=palette, line_width=2, show_scalar_bar=False)

if SHOW_HLA_PLANE:    
    plotter.add_mesh(hla_plane, color=palette[idx],  opacity=0.3, show_edges=False, line_width=2)
    plotter.add_mesh(hla_edges, color=dark_palette[idx], line_width=1)

plotter.view_isometric()
plotter.enable_parallel_projection()

# plotter.camera.position = (hla_support+hla_normal).tolist()
# plotter.camera.focal_point = hla_support.tolist()
# plotter.camera.up = (0.0, 1.0, 0.0)
plotter.camera.zoom(1.2)

plotter.show(
    window_size=[1200,1200], 
    # jupyter_backend='static',
    cpos=CPOS
)

In [ ]:
plotter.camera_position